In [36]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.neural_network import MLPClassifier, MLPRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.externals import joblib

In [37]:
train = pd.read_csv("../data/cars/train_data.csv", sep = "\t", encoding='utf-8')
test = pd.read_csv("../data/cars/test_data.csv", sep = "\t", encoding='utf-8')

In [38]:
y = train['price']
test_y = test['price']
X = train.drop(['price'], axis = 1)
test_X = test.drop(['price'], axis = 1)

#### Multilabel Encoding

In [39]:
def f(x, l):
    if type(x) is list:
        l.update(x)
class MultilabelEncoding(BaseEstimator, TransformerMixin):
    def __init__(self, col_name):
        self.col_name = col_name
        self.type_list = set()
    def fit(self, X_df, y=None):
        types_sr = X_df[self.col_name].str.replace('[/+]', ' ').str.split()
        types_sr.apply(f, args=(self.type_list,))
        self.type_list = list(self.type_list)
        return self
    def transform(self, X_df, y=None):
        transformed_df = X_df.copy()
        for i in range(len(self.type_list)):
            transformed_df[self.col_name + '_' + self.type_list[i]] = transformed_df[self.col_name].apply(lambda x: 1 if ((type(x) is str) and (self.type_list[i] in x)) else 0)
        transformed_df.drop(self.col_name, axis=1, inplace=True)
        return transformed_df

#### col add and drop

In [40]:
class ColAdderDropper(BaseEstimator, TransformerMixin):
    def __init__(self, num_top_brands = 10):
        # TODO
        self.num_top_brands = num_top_brands
        self.dropped_cols = ['url', 'name', 'model', 'weightTotal', 'fuelType', 'vehicleTransmission', 'modelDate']
    def fit(self, X_df, y=None):
        brand_col = X_df.brand.str.extract(r'([a-zA-z]+)', expand=False)
        self.brand_counts_ = brand_col.value_counts()
        brands = list(self.brand_counts_.index)
        self.top_brands_ = brands[:max(1, min(self.num_top_brands, len(brands)))]
        return self
    def transform(self, X_df, y=None):
        df = X_df.copy()
        brand_col = df.brand.str.extract("([a-zA-z]+)", expand=False)
        brand_col[~brand_col.isin(self.top_brands_)] = 'Others'
        df["brand"] = brand_col
        df.drop(self.dropped_cols, axis=1, inplace=True)
        return df

In [41]:
cat_multi = ['vEfuelType', 'driveWheelConfiguration']
cat_single = ['brand', 'eLabel', 'bodyType', 'vEengineType']

In [42]:
col_adderdropper = ColAdderDropper()

In [43]:
num_cols = []
for col in X.columns:
    if (not col in cat_single) and (not col in cat_multi) and (not col in col_adderdropper.dropped_cols):
        num_cols.append(col)

#### Hàm train và validate model

In [44]:
def train_and_val(full_pipeline, train_X, train_y, val_X, val_y):
    full_pipeline.fit(train_X, train_y);
    #print("n_brands:", len(col_adderdropper.top_brands_))
    pred_y = full_pipeline.predict(val_X)
    train_score = full_pipeline.score(train_X, train_y)
    val_score = full_pipeline.score(val_X, val_y)
    print(train_score, val_score)
    print("MSE =", np.round(np.mean((pred_y - val_y) ** 2), 3))
    print("MAE =", np.round(np.mean(np.abs(pred_y - val_y)), 3))
    return train_score, val_score

### các pipeline cần thiết cho pipeline cho preprocessing

In [45]:
imp_mean = SimpleImputer(strategy='mean') 
imp_mode = SimpleImputer(strategy='most_frequent')

encoding = OneHotEncoder(handle_unknown='ignore', sparse=False)
vEfuelType_encoder = MultilabelEncoding('vEfuelType')
driveWheelConfig_encoder = MultilabelEncoding('driveWheelConfiguration')

categorical_transformer = make_pipeline(imp_mode, encoding)

colTransform = ColumnTransformer(transformers=[('numerical', imp_mean, num_cols),\
                                               ('categorical', categorical_transformer, cat_single),
                                               ('vEfuelType', vEfuelType_encoder, ['vEfuelType']),
                                               ('driveWheelConfig', driveWheelConfig_encoder, ['driveWheelConfiguration'])])

colNormalize = StandardScaler()

#preprocessing = make_pipeline(col_adderdropper, colTransform, colNormalize)

### Full pipeline với MLPRegressor

In [ ]:
best_n_brands = 80
mlpregressor = MLPRegressor(hidden_layer_sizes=(256, 512, 512, 256, ), solver='adam', learning_rate='adaptive'\
                            ,random_state=0, max_iter=500, early_stopping=True, verbose=1)
full_pipeline = make_pipeline(col_adderdropper, colTransform, colNormalize, mlpregressor)
full_pipeline.set_params(coladderdropper__num_top_brands=best_n_brands)
train_and_val(full_pipeline, X, y, test_X, test_y)

Iteration 1, loss = 96.64648980
Validation score: 0.786945
Iteration 2, loss = 45.67938600
Validation score: 0.887709
Iteration 3, loss = 36.19118507
Validation score: 0.922643
Iteration 4, loss = 33.56245111
Validation score: 0.933794
Iteration 5, loss = 32.40656744
Validation score: 0.936253
Iteration 6, loss = 28.02480550
Validation score: 0.942617
Iteration 7, loss = 27.26085175
Validation score: 0.927227
Iteration 8, loss = 23.58364638
Validation score: 0.945869
Iteration 9, loss = 23.53042504
Validation score: 0.946286
Iteration 10, loss = 20.96019146
Validation score: 0.922613
Iteration 11, loss = 28.00011589
Validation score: 0.959852
Iteration 12, loss = 18.65896272
Validation score: 0.962853
Iteration 13, loss = 17.20015540
Validation score: 0.965803
Iteration 14, loss = 17.95327746
Validation score: 0.952739
Iteration 15, loss = 19.66866823
Validation score: 0.871601
Iteration 16, loss = 19.85778606
Validation score: 0.965690
Iteration 17, loss = 17.68356495
Validation score

#### save model

In [16]:
joblib.dump(full_pipeline, 'nn_80_final.pkl')

['nn_80_final.pkl']

### full pipeline với RandomForestRegressor

In [47]:
rfregressor = RandomForestRegressor(n_estimators=1024, random_state=0, verbose=1)
full_pipeline = make_pipeline(col_adderdropper, colTransform, colNormalize, rfregressor)
full_pipeline.set_params(coladderdropper__num_top_brands = 90)
train_and_val(full_pipeline, X, y, test_X, test_y)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


KeyboardInterrupt: 

In [19]:
# save model
# joblib.dump(full_pipeline, 'rf_1024.pkl')